In [2]:
search_for = 130
start_from = 0
threads = 10
thread = 3
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.002334117889404297
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 125
all cases: 749112551
type: [1, 1, 1, 1, 130] 130
cases of this type: 285610000
100000 33.020399834698004
200000 55.50262188611315
300000 57.15979946721167
400000 55.13169115772937
500000 57.198253629069974
600000 59.12804107134768
700000 56.17959621323507
800000 55.485721891989385
900000 54.68889017752102
1000000 55.79445837832927
1100000 39.39923291783448
1200000 33.11921510162998
1300000 54.65449796287616
1400000 54.70931597482791
1500000 53.6895061808796
1600000 55.301783204512084
1700000 59.25930929586693
1800000 55.05259569481935
1900000 54.87728804991873
2000000 56.890645121086976
2100000 55.76674798431469
2200000 42.87026712793732
2300000 29.05001399230538
2400000 54.04535896772961
2500000 54.06423521398059
2600000 52.82280323772881
2700000 54.03195730511005
2800000 55.87618365814863
2900000 53.147144174617665
3000000 52.7897049140

29700000 19.77250444880966
29800000 50.41350818892643
29900000 53.438860331134904
30000000 53.84896277343073
30100000 58.92375873071851
30200000 55.21566009691039
30300000 57.377626135703615
30400000 60.5935912130983
30500000 56.08798906857549
30600000 23.620037136760057
30700000 58.44640755808777
30800000 59.3307762752329
30900000 60.32280083909108
31000000 41.47569759402497
31100000 39.59540609885523
31200000 57.18349106624194
31300000 57.635236453279916
31400000 56.76285886292279
31500000 54.88933917972413
31600000 56.0710047972125
31700000 52.125365632234534
31800000 52.37847025948429
31900000 18.741054240273165
32000000 52.107488902473634
32100000 53.23313610173538
32200000 53.776226496335774
32300000 55.349979022322415
32400000 57.17836692825714
32500000 56.06905869550147
32600000 58.227575509988775
32700000 49.588221524244744
32800000 31.41009475651071
32900000 56.3598298264152
33000000 60.255844279003774
33100000 66.47282035513192
33200000 51.19757945233797
33300000 35.51662144

59900000 44.426812249797926
60000000 54.580715819057964
60100000 52.606233872286595
60200000 49.830333718750786
60300000 51.52690759968227
60400000 42.94622084972497
60500000 22.867322002433163
60600000 50.47782245312031
60700000 49.78371906333304
60800000 50.65930195968355
60900000 49.74528962932717
61000000 54.89409099549534
61100000 24.948668478330536
61200000 52.02939612840379
61300000 59.24506705913049
61400000 56.7694854143377
61500000 59.61908183532467
61600000 56.02614047166735
61700000 58.57915959111217
61800000 57.358390538153465
61900000 57.37892959828862
62000000 37.41271367299279
62100000 37.494654217932535
62200000 53.42727327090121
62300000 50.83363186703934
62400000 48.11474705081915
62500000 50.7340305197597
62600000 41.577721589738985
62700000 23.884479443460968
62800000 49.38122063489413
62900000 48.9949326178979
63000000 49.471489175688205
63100000 49.911863792945624
63200000 53.272179486921495
63300000 22.580764166315518
63400000 55.05960477319614
63500000 57.75406

90000000 49.77417158234672
90100000 49.57746778636369
90200000 52.90831003052938
90300000 53.99482196336536
90400000 51.15475795176854
90500000 53.54164094646751
90600000 50.79596265309665
90700000 47.2909827021537
90800000 24.498805810293614
90900000 41.14360218834446
91000000 48.4091597637973
91100000 47.884133864596926
91200000 24.704409118214866
91300000 39.02850515426514
91400000 47.4517072274348
91500000 47.02515689439527
91600000 30.162662132526712
91700000 36.66187765596127
91800000 48.4707391409574
91900000 49.237414575722305
92000000 51.646942379424466
92100000 52.37091258425739
92200000 53.06205256750522
92300000 52.17782826298277
92400000 52.52942675515524
92500000 52.23633937057193
92600000 53.431320864233236
92700000 49.47197582489663
92800000 48.813825989097374
92900000 46.808601036155885
93000000 29.766883476751406
93100000 35.374997363533524
93200000 45.14941695404078
93300000 47.38873716788898
93400000 22.70328609630803
93500000 38.949904214098396
93600000 47.52926153

119400000 45.22682670527336
119500000 45.9219470949654
119600000 18.617196370986782
119700000 45.22754473313926
119800000 13.165184272488661
119900000 42.05915569696518
120000000 23.531912971027996
120100000 43.17408321638112
120200000 46.1186070168742
120300000 46.68573968103271
120400000 45.55171249158347
120500000 47.13322627373799
120600000 44.96861889201094
120700000 49.36795266410457
120800000 51.3251438256945
120900000 53.18534885646103
121000000 58.282391166900595
121100000 53.13860328339852
121200000 51.54440085439145
121300000 52.48439699178313
121400000 50.212792777610865
121500000 53.19422958230996
121600000 49.726495116609776
121700000 49.59364954336986
121800000 23.106762586099986
121900000 46.41477448448709
122000000 14.413723521702346
122100000 39.45575992291286
122200000 31.19820931723569
122300000 49.305289900091054
122400000 49.86871016158134
122500000 52.2899460001288
122600000 50.502456696227014
122700000 53.6433696992502
122800000 50.35336348961156
122900000 54.18

...

638700000 1.0525914390677804
638800000 0.8978303098916544
638900000 1.364784506392884
639000000 1.2755549254346272
639100000 2.075639288423301
639200000 1.1336745545467535
639300000 2.006650262368078
best so far: 0
type: [1, 1, 2, 13, 5] 130
cases of this type: 5492500
639400000 2.7870554257358604
639500000 2.3924452447432816
639600000 2.2869701010963603
639700000 1.6505111351595996
639800000 1.1215450897613883
639900000 0.9381032876642651
640000000 0.9713413248195687
640100000 1.0121313199672004
640200000 0.9463633223097576
640300000 1.3161132805365932
640400000 0.9738740248988748
640500000 1.3974233598759491
640600000 1.3107631836755984
640700000 1.7347497238298355
640800000 1.0425780016074477
640900000 1.399554839186742
641000000 1.1949009970180093
641100000 1.679227069552056
641200000 1.657684907343451
641300000 1.666407870512855
641400000 1.7951273618545973
641500000 0.9912455559831289
641600000 2.3967810827355627
641700000 1.829952162057168
641800000 2.308047249708444
641900000 1

665500000 0.36210829258155297
665600000 0.3655454034004589
665700000 0.34247467678556176
665800000 0.33853224431440504
665900000 0.34534585432446
666000000 0.34120226187927916
666100000 0.5559767046702166
666200000 0.5956310393227439
666300000 0.5939284685878621
666400000 2.1679328372735767
666500000 2.4921156104518634
666600000 2.199466894531474
666700000 1.724173543224056
666800000 1.786727008849776
666900000 2.4649326877015927
667000000 1.9337898221605891
667100000 1.7973571442425589
667200000 1.7562019473719317
best so far: 0
type: [1, 1, 10, 13, 1] 130
cases of this type: 219700
667300000 0.7143775232432285
667400000 0.28820348605235613
best so far: 0
type: [1, 1, 13, 1, 10] 130
cases of this type: 1690000
667500000 0.8728274635484895
667600000 1.4088902406706227
667700000 1.3307567507698774
667800000 1.5897382649738645
667900000 2.770231250667306
668000000 1.014865820368574
668100000 0.3366951972656985
668200000 0.33605259716245534
668300000 0.426568776482286
668400000 0.35943592

691300000 1.210878136737672
691400000 1.2906222873676538
691500000 1.3851123877041738
691600000 1.1803291564332565
691700000 0.8827838408549329
691800000 0.6374445143231319
691900000 1.314700426283033
692000000 1.2939616802710534
692100000 1.3306849500168945
692200000 1.229790581247263
692300000 0.633068140042605
692400000 1.4043927323399028
692500000 1.3189428267605539
692600000 1.4473635617826819
692700000 1.3071340059653296
692800000 0.9279728563315418
692900000 0.6738950147088586
693000000 1.307374724237414
693100000 1.383079211722505
693200000 1.2751773000792463
693300000 1.0818511588690594
693400000 0.9126264575421208
693500000 0.9649593478883227
693600000 1.2619971665917542
693700000 1.3708706521747986
693800000 1.2341078890470922
693900000 0.9130868914215061
694000000 0.7239592786269837
694100000 1.300385082471166
694200000 1.335934580079354
694300000 1.402055116307193
694400000 0.9136237390121897
694500000 0.8928115375920513
694600000 0.9289719055538045
694700000 1.43170272964

718600000 0.23258170566693215
best so far: 0
type: [1, 2, 5, 13, 1] 130
cases of this type: 109850
718700000 0.24276219819755948
best so far: 0
type: [1, 2, 13, 1, 5] 130
cases of this type: 211250
718800000 0.09927016331292722
718900000 0.11105813320882718
719000000 0.11081604356868864
best so far: 0
type: [1, 2, 13, 5, 1] 130
cases of this type: 42250
best so far: 0
type: [1, 2, 65, 1, 1] 130
cases of this type: 8450
best so far: 0
type: [1, 5, 1, 1, 26] 130
cases of this type: 2284880
719100000 0.21434884615902702
719200000 0.31882082283708124
719300000 0.282749213431553
719400000 0.29194816272678575
719500000 0.27289909082083236
719600000 0.27997176569924354
719700000 0.28383238142558403
719800000 0.26564057117080625
719900000 0.24807728493933082
720000000 0.29918368278403346
720100000 0.15922686461449595
720200000 0.10660942986603976
720300000 0.12228450370681751
720400000 0.10529246908639603
720500000 0.2008628354217867
720600000 0.26806932757857377
720700000 0.24715812460770809


738700000 0.13906892283288902
738800000 0.1528433110710025
738900000 0.17321966011155313
739000000 0.15239285479936135
739100000 0.17081651754018268
739200000 0.18678577436581054
739300000 0.16041440460951592
739400000 0.17419687148338955
739500000 0.14970100185910068
739600000 0.13888794544432428
739700000 0.1364566627809763
739800000 0.13519516886570387
739900000 0.14265907290588684
740000000 0.14513927671861251
740100000 0.12957172612912457
740200000 0.1382605059971498
740300000 0.15651195014412603
740400000 0.13644518293927643
740500000 0.13554721786322926
740600000 0.17730283463433982
740700000 0.14797054520318442
740800000 0.12738834023462534
740900000 0.12409091114186643
741000000 0.13954382149084343
741100000 0.117079945935299
741200000 0.12734455236595074
741300000 0.12329315226872564
741400000 0.14068061407383878
741500000 0.11646563034011723
741600000 0.11966958105452723
741700000 0.15116557162807254
741800000 0.13029980339504876
best so far: 0
type: [2, 1, 1, 5, 13] 130
cas